In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
def update_section_name(section_name):
    if "data sheet" in section_name.lower():
        section_name = "Data sheet"
    elif "application note" in section_name.lower():
        section_name = "Application note"
    elif "technical note" in section_name.lower():
        section_name = "Technical note"
    elif "manual" in section_name.lower():
        section_name = "Manual"
    elif "article" in section_name.lower():
        section_name = "Article"
    elif "end-user documentation" in section_name.lower():
        section_name = "End-user documentation"
    else:
        section_name = section_name
    return section_name

In [12]:
different_section_names = []


In [13]:
df = pd.read_excel("./data/AI_descriptions_all (1).xlsx")


In [14]:
def find_downloads(url):
    section_links = {}
    response = requests.get(url)
    html = response.text

    # Parse the HTML
    soup = BeautifulSoup(html, 'html.parser')
    # Dictionary to store the result

    # Find all section subtitles and corresponding links

    for subtitle in soup.find_all("h3", class_="Section__subtitle"):
        section_name = subtitle.text.strip()
        section_name = update_section_name(section_name)
        # print(subtitle.parent())
        if section_name == 'Stay in touch for product information and software updates':
            break
        if section_name not in different_section_names:
            different_section_names.append(section_name)

        downloads_name = [span.text.strip() for span in subtitle.find_next_sibling("ul").find_all("span")]
        
        links = [a['href'] for a in subtitle.find_next_sibling("ul").find_all("a", href=True)]
        links = [link for link in links if not link.endswith(".aspx")]
        hrefs = {name: 'https://www.kongsberg.com' + link if link.startswith('/') else link for name, link in zip(downloads_name, links)}
        # print(url)

        section_links[section_name] = hrefs
        # print(section_links)

    for section in soup.find_all("section"):
        title_tag = soup.find('h2', class_='Section__title', text='Additional product information')
        download_tag = soup.find('h2', class_='Section__title', text='Downloads')
        if title_tag:
            section_name = title_tag.text.strip()
            if section_name not in different_section_names:
                different_section_names.append(section_name)            
            links = []
            for link_tag in section.find_all("a", href=True):
                link_href = link_tag['href']
                if link_href.startswith("/"):
                    link_href = 'https://www.kongsberg.com/' + link_href
                links.append(link_href)
                downloads_itemname = link_tag.find("span")
            section_links[section_name] = links
        elif download_tag:
            section_name = download_tag.text.strip()
            if section_name not in different_section_names:
                different_section_names.append(section_name)
            links = []
            for link_tag in section.find_all("a", href=True):
                link_href = link_tag['href']
                if link_href.startswith("/"):
                    link_href = 'https://www.kongsberg.com/' + link_href
                if not link_href.endswith(".aspx"):
                    links.append(link_href)
                downloads_itemname = link_tag.find("span")
            section_links[section_name] = {downloads_itemname: links}
    if not url:
        links = {"Data Sheet": df[df['url'].isnull()]['Data sheets']}
        print(links)
        section_links["Data sheet"] = links
    return section_links


In [15]:
df["downloads"] = df.apply(lambda row: find_downloads(row["url"]) if pd.notna(row["url"]) else {"Data sheet": {"Data sheet": [row["Data sheets"]]}}, axis=1)


C:\Users\HerborgHermansenTvei\AppData\Local\Temp\ipykernel_27036\4245002080.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  title_tag = soup.find('h2', class_='Section__title', text='Additional product information')
C:\Users\HerborgHermansenTvei\AppData\Local\Temp\ipykernel_27036\4245002080.py:33: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  download_tag = soup.find('h2', class_='Section__title', text='Downloads')


In [16]:
df

,Product_Name,Product category,General text,Features,Technical Specifications,url,Data sheets,downloads
0,AIS physical shore stations,"Communication solutions, Naval",NaN,NaN,NaN,https://www.kongsberg.com/maritime/products/on...,NaN,"{'Brochure': {}, 'Data sheet': {'AIS Base Stat..."
1,AIS service management system,Surveillance & monitoring,Monitoring of all AIS vessels and AIS AtoN wit...,- Monitoring of all AIS vessels and AIS AtoN ...,I don't know,https://www.kongsberg.com/maritime/products/on...,NaN,{}
2,AIS space receivers,Surveillance & monitoring,NaN,NaN,NaN,https://www.kongsberg.com/maritime/products/on...,NaN,{'Data sheet': {'ASR c50': 'https://www.kongsb...
3,APOS - Operating station for HiPAP and HPR Aco...,Surveillance & monitoring,The product brings significant value to users ...,- Innovative design\n - Advanced functionalit...,- Category: Design\n - Parameters: Innovati...,https://www.kongsberg.com/maritime/products/Ac...,https://www.kongsberg.com/globalassets/maritim...,{'Application note': {'Transducer alignment/Hi...
4,APOS Survey - Surveyor's independent Operator ...,Communication solutions,"APOS Survey offers full LBL and SSBL, quicker ...",APOS Survey offers: \n- Full LBL and SSBL \n- ...,"MP8300 computer: Type-approved by ABS, BV, CCS...",https://www.kongsberg.com/maritime/products/Ac...,NaN,{}
...,...,...,...,...,...,...,...,...
224,MRU E,"Geophysical survey, Seafloor mapping, Surveill...",The MRU E is an extended temperature motion se...,- 0.05° roll and pitch accuracy\n- Outputs re...,"MRU E, ORIENTATION OUTPUT INPUT FORMATS\n- Ang...",NaN,https://www.kongsberg.com/contentassets/77678b...,{'Data sheet': {'Data sheet': ['https://www.ko...
225,MRU 5,"Ocean science, Surface navigation & positioning",THE IDEAL MARINE MOTION SENSOR. The fifth gene...,• 0.02° roll and pitch accuracy. • Outputs on ...,MRU 5. ORIENTATION OUTPUT: Analog channels (ju...,NaN,https://www.kongsberg.com/contentassets/77678b...,{'Data sheet': {'Data sheet': ['https://www.ko...
226,MRU H,"Ocean science, Surface navigation & positioning",THE VERSATILE HEAVE COMPENSATOR This fifth gen...,• 0.05° roll and pitch accuracy • High accurac...,MRU H ORIENTATION Angular orientation range ±1...,NaN,https://www.kongsberg.com/contentassets/77678b...,{'Data sheet': {'Data sheet': ['https://www.ko...
227,MRU 5+,"Ocean science, Surface navigation & positioning",The MRU 5+ product is upgraded with the best M...,• 0.008° roll and pitch accuracy\n• Exceptiona...,MRU 5+ MK-II\nORIENTATION OUTPUT Analog channe...,NaN,https://www.kongsberg.com/contentassets/77678b...,{'Data sheet': {'Data sheet': ['https://www.ko...


In [17]:
df["url"].iloc[2]

'https://www.kongsberg.com/maritime/products/onshore/space-based/ais-space-receiver/'

In [19]:
df.to_excel("./data/downloads.xlsx")